## DS 5460 Big Data Scaling, SPRING 2024

## Homework Assignment \#2: Naive Bayes in MapReduce

Objectives:
- Build proficiency with MapReduce concepts like mappers, reducers, and parallelization as they apply to a machine learning algorithm implementation
- Understand the Naive Bayes algorithm and how it can be parallelized effectively using the MapReduce framework

Instructions:

In this assignment you will implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your algorithm, you'll first test it on a small dataset. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. 

Be sure to read all text cell descriptions and comments closely to fill in your solution when expected. In addition to the specified tasks, there may be `TODO` comments, for you to complete to set up your environment properly.

We will only grade code written in the designated spaces. If a question's instructions are unclear, please reach out for clarification on Piazza. We expect each student to write their own code independently. If GenAI tools are used, **in which specific questions, commands, code, etc,. and how they were used must be disclosed properly in a separate text cell at the end of this notebook. Remember that the short answer solutions must be your independent work and should not be derived from GenAI tools.** 

__TIPS:__ 
1. Make use of your peers and TAs by asking questions on Piazza. Everyone has different experiences and background so don't be shy; all questions are welcome!
2. Only use ONE (1) reducer for this assignment.




## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [1]:
!hadoop version

Hadoop 2.10.2
Subversion Unknown -r Unknown
Compiled by bigtop on 2023-09-02T20:18Z
Compiled with protoc 2.5.0
From source with checksum 4bb37aedd62b388ba458183bdd93130
This command was run using /usr/lib/hadoop/hadoop-common-2.10.2.jar


In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
# TODO: setting up global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HDFS_DIR = "/user/root/HW2"
# HOME_DIR = "/media/notebooks/Assignments/HW2"
# %cd {HOME_DIR}

In [4]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [7]:
# TODO: path to dataset - ADJUST AS NEEDED
ENRON = "email.txt"

In [8]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir -p {HDFS_DIR}
!hdfs dfs -ls 

Found 1 items
drwxr-xr-x   - root hadoop          0 2024-02-07 02:08 HW2


# About the Data
The main task is to train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/email.txt`__). (The original data is available [here](https://www2.aueb.gr/users/ion/data/enron-spam/), which was created by researchers working on personalized Bayesian spam filters.)

__Format:__   
All messages are saved in a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

Data dictionary:  
>   `ID: (string) unique message identifier`  
    `SPAM: (binary) with 1 indicating a spam message and 0 indicating a ham (legitimate) message`  
    `SUBJECT: (string) title of the message`  
    `CONTENT: (string) content of the message`   
    
Note that `SUBJECT` or `CONTENT` may be "NA", and all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [9]:
!pwd

/


In [10]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
!head -n 5 {ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [11]:
# check how many messages/lines are in the file 
# (Note: this number may be off by 1 if the last line doesn't end with a newline)
!wc -l {ENRON}

100 email.txt


In [12]:
# load the data into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal {ENRON} {HDFS_DIR}/enron.txt

In [13]:
!hdfs dfs -ls {HDFS_DIR}

Found 1 items
-rw-r--r--   1 root hadoop     204559 2024-02-07 02:08 /user/root/HW2/enron.txt


# Question 1:  Enron Ham/Spam EDA
Before building the classifier, we first need to explore this data to find out which words occur more in spam emails than in legitimate ("ham") emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Tasks:
* __a) Implement mapper and reducer scripts:__ Complete the missing components of the code in __`eda/mapper.py`__ and __`eda/reducer.py`__ to create a Hadoop MapReduce job that counts how many times each word in the corpus occurs in an email for each class (spam vs ham). Pay close attention to the data format specified in the comments of these scripts. 

* __b) Unit test your scripts and run an MR job:__ Create unit tests to confirm that your code works as expected, then write the Hadoop Streaming command to apply your analysis to the actual Enron data. Save the results to a local file.

* __c) Code in Notebook and answer below:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`part b`' and then report the answer in the Student Answers area provided below.)

* __d) Code in Notebook and answer below:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file. Then describe in words below how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reordering the records from `part a`, you should be able to just use `/bin/cat` for both_. However, if the sorted output is not what you expect, you can duplicate your mapper.py as a mapper-by-class.py file and modify the emitted output like we did in class with the `Topwords` exercise, where we reordered the output to print count then word)


### Q1 Student Answers:
> __c)__ assistance appears in spams 8 times and in hams 2 times.

> __d)__ I can use commands: 

! cat sort_results.txt | sort -k2,2 -k3,3nr | head -n 10

! cat sort_results.txt | sort -k2,2r -k3,3nr | head -n 10

In [19]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x mapper.py
!chmod a+x reducer.py

In [27]:
# part a - unit test eda/mapper.py
# ID \t SPAM \t SUBJECT \t CONTENT \n
# You may use the following test case or create your own 
!echo "d1	1	title	body\nd2	0	title	body" | /mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [53]:
# part a - unit test eda/reducer.py
# word \t class \t partialCount 
# You may use the following test case or create your own 

!echo "one	1	1\none	1	1\none	1	1\none	0	1\ntwo	0	1"  | /reducer.py

one	1	3
one	0	1
two	0	1


In [54]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [55]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hadoop jar {JAR_FILE} \
  -files /reducer.py,/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.10.2.jar] /tmp/streamjob4796811118056949793.jar tmpDir=null
24/02/07 04:03:01 INFO client.RMProxy: Connecting to ResourceManager at hw2hw2cluster-m/10.128.0.2:8032
24/02/07 04:03:01 INFO client.AHSProxy: Connecting to Application History server at hw2hw2cluster-m/10.128.0.2:10200
24/02/07 04:03:02 INFO client.RMProxy: Connecting to ResourceManager at hw2hw2cluster-m/10.128.0.2:8032
24/02/07 04:03:02 INFO client.AHSProxy: Connecting to Application History server at hw2hw2cluster-m/10.128.0.2:10200
24/02/07 04:03:02 INFO mapred.FileInputFormat: Total input files to process : 1
24/02/07 04:03:03 INFO mapreduce.JobSubmitter: number of splits:9
24/02/07 04:03:04 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1707263943154_0004
24/02/07 04:03:04 INFO conf.Configuration: resource-types.xml not found
24/02/07 04:03:04 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
24/02/07 04:03:04 INFO resource.Res

In [56]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > results.txt

In [57]:
# part b - write your grep command here
!grep "assistance" results.txt

assistance	1	8
assistance	0	2


In [74]:
# part d - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [75]:
# part d - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2n -k3,3nr" \
  -mapper /bin/cat \
  -input {HDFS_DIR}/eda-output/part-0000* \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.10.2.jar] /tmp/streamjob735695789868908910.jar tmpDir=null
24/02/07 04:37:13 INFO client.RMProxy: Connecting to ResourceManager at hw2hw2cluster-m/10.128.0.2:8032
24/02/07 04:37:14 INFO client.AHSProxy: Connecting to Application History server at hw2hw2cluster-m/10.128.0.2:10200
24/02/07 04:37:14 INFO client.RMProxy: Connecting to ResourceManager at hw2hw2cluster-m/10.128.0.2:8032
24/02/07 04:37:14 INFO client.AHSProxy: Connecting to Application History server at hw2hw2cluster-m/10.128.0.2:10200
24/02/07 04:37:15 INFO mapred.FileInputFormat: Total input files to process : 3
24/02/07 04:37:15 INFO mapreduce.JobSubmitter: number of splits:10
24/02/07 04:37:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1707263943154_0008
24/02/07 04:37:16 INFO conf.Configuration: resource-types.xml not found
24/02/07 04:37:16 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
24/02/07 04:37:16 INFO resource.Res

In [76]:
!hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000* > sort_results.txt

In [79]:
! cat sort_results.txt | sort -k2,2 -k3,3nr | head -n 10

the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135	
sort: write failed: 'standard output': Broken pipe
sort: write error


In [80]:
! cat sort_results.txt | sort -k2,2r -k3,3nr | head -n 10

the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153	
sort: write failed: 'standard output': Broken pipe
sort: write error


# Question 2: Document Classification Task Overview
In this question we'll use the following example to 'train' a LaPlace (plus one) smoothed Multinomial Naive Bayes model and classify a test document. In addition to the lecture slides, you may also want to review [this chapter on text classification and Naive Bayes](https://nlp.stanford.edu/IR-book/pdf/13bayes.pdf), which provides a thorough introduction to the document classification task and the math behind Naive Bayes. 


### Tasks:

* __a) spam email probability calculation:__ Below in the code cell labeled `part a)`, calculate the probability that an email is spam given that it contains only one word, "Urgent", from the vocabulary, using the provided probabilities. Additionally, spam is $40\%$ of all email, and $80\%$ of spam email contains the word “Urgent”, i.e, $Pr(Urgent|SPAM) =0.8.$


* __b) calculate probabilities for the classifier:__ Fill in the missing code in code cell labeled `part b)` to compute the probabilities used in the Multinomial Naive Bayes classifier with Laplace (plus one) smoothing. Use the following training dataset of 5 documents for a 2 Class problem: HAM versus SPAM.

**Training Data**

|DocId |Class | Document
|---|---|---|
|d1 | HAM | good
|d2 | SPAM | very good
|d3 | SPAM | good bad
|d4 | HAM | very bad
|d5 | SPAM | very bad very good


The vocabulary of the dataset is [good, very, bad]. 

* __c) Predict on the test data:__ Complete the code cell labeled `part c` to classify the test data )consisting of a single test case)

__Test Data__

| DocId | Class | Document String
|---|---|---|
| d6 | ?? | good bad very

-----------------------------------

* __d) Short answer below:__ Equation 13.3 in [this chapter on text classification and Naive Bayes](https://nlp.stanford.edu/IR-book/pdf/13bayes.pdf) shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate $\hat{P}(c|d)$? Your answer should include both mathematical notatation and short written explanation.




### Q2 Student Answers:

> __d)__ 
$c$ = $argmax$$\hat{P}(c|d)$ = $argmax$$\hat{P}(c)\space\Pi{P}(t_i|c)$

$\hat{P}(c)$ which is Prior probability of hypothesis, and ${P}(t_i|c)$ which is the likehood are required.

For each $\hat{P}(c_i)$, select the class $c_k$ making the largest $\hat{P}(c_i)$.




In [82]:
# part a) TODO: Complete the calculation for Pr(SPAM|X=Urgent)
# Given probabilities
pr_urgent = 0.5  # Pr("Urgent")
pr_spam = 0.4  # Pr(SPAM)
pr_urgent_given_spam = 0.8  # Pr(Urgent|SPAM)

# TODO: your code here to calculate Pr(SPAM|X=Urgent)
pr_spam_given_urgent = pr_spam * pr_urgent_given_spam / pr_urgent
print(f'Pr(SPAM|X=Urgent) = {pr_spam_given_urgent}')

Pr(SPAM|X=Urgent) = 0.6400000000000001


In [85]:
# part b

import pandas as pd
import numpy as np

vocabulary = ["bad", "good", "very"]

''' Here's the data again that we are representing below
|DocId |Class | Document
|---|---|---|
|d1 | HAM | good
|d2 | SPAM | very good
|d3 | SPAM | good bad
|d4 | HAM | very bad
|d5 | SPAM | very bad very good
'''

# Document by word matrix
doc_by_word = np.array([[0, 1, 0],
                        [0, 1, 1],
                        [1, 1, 0],
                        [1, 0, 1],
                        [1, 1, 2]])

# y_train: 0 for Ham and 1 for Spam
class_by_doc = np.array([0, 1, 1, 0, 1])
df = pd.DataFrame(np.c_[class_by_doc, doc_by_word], index = ["d1", "d2", "d3", "d4", "d5"], columns = ["Class"] + vocabulary)
display(pd.DataFrame(np.c_[class_by_doc, doc_by_word], index = ["d1", "d2", "d3", "d4", "d5"], columns = ["Class"] + vocabulary))

,Class,bad,good,very
d1,0,0,1,0
d2,1,0,1,1
d3,1,1,1,0
d4,0,1,0,1
d5,1,1,1,2


In [87]:
# Learn the Naïve Bayes Classification:
model_priors = df['Class'].value_counts(normalize=True)
print(f"model_priors: {model_priors}") # Expected output - model_priors: [0.4 0.6]

model_priors: 1    0.6
0    0.4
Name: Class, dtype: float64


In [95]:
# Calculate Pr(word_i|ham) or HAM class conditional probability, using LaPlace Smoothing
print(doc_by_word[class_by_doc == 0, :])
total_words_given_ham = np.sum(doc_by_word[class_by_doc == 0, :])
model_data_given_ham = (np.sum(doc_by_word[class_by_doc == 0, :],axis = 0)+1)/(total_words_given_ham+len(vocabulary))
print(f"Pr(word_i|ham):  {np.round(model_data_given_ham, 3)}")

[[0 1 0]
 [1 0 1]]
Pr(word_i|ham):  [0.333 0.333 0.333]


In [96]:
# Calculate Pr(word_i|spam) aka SPAM class conditionals:
print(doc_by_word[class_by_doc == 1, :])
total_words_given_spam = np.sum(doc_by_word[class_by_doc == 1, :])
model_data_given_spam = (np.sum(doc_by_word[class_by_doc == 1, :],axis = 0)+1)/(total_words_given_spam+len(vocabulary))
print(f"Pr(word_i|spam):  {np.round(model_data_given_spam, 3)}")

[[0 1 1]
 [1 1 0]
 [1 1 2]]
Pr(word_i|spam):  [0.273 0.364 0.364]


In [97]:
# part c

# Test document terms are: bad, good, very
d6 = [1, 1, 1] # TEST DOCUMENT
display(pd.DataFrame([d6], index = ["d6"], columns = vocabulary))

# Naïve Bayes Classification
# Likelihood
# Applying the Unigram (single word) Language Model
likelihood_d6_given_ham = np.prod(model_data_given_ham)
likelihood_d6_given_spam = np.prod(model_data_given_spam)

# Calculate Posterior Probabilities using the learned Naive Bayes Model
pr_ham =  0.4 * likelihood_d6_given_ham
pr_spam = 0.6 * likelihood_d6_given_spam

# Printing the normalized posterior probabilities
print(f"Posterior Probabilities in % is: Pr(Ham|D6) is : {100 * pr_ham / (pr_spam + pr_ham):7.0f}")
print(f"Posterior Probabilities in % is: Pr(SPAM|D6) is : {100 * pr_spam / (pr_spam + pr_ham):7.0f}")

,bad,good,very
d6,1,1,1


Posterior Probabilities in % is: Pr(Ham|D6) is :      41
Posterior Probabilities in % is: Pr(SPAM|D6) is :      59


# Question 3: Naive Bayes Inference
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the example corpus from [this chapter on text classification and Naive Bayes](https://nlp.stanford.edu/IR-book/pdf/13bayes.pdf) to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Tasks:
* __a) short answer and complete `part a` code cell:__ run the provided cells to create the example files and write the command to load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the keys and values in this file, and what do the fields represent?

* __b) code mapper:__ Complete the code in __`model/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected.

* __c) run a Hadoop job:__ Write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:`_You'll need to load the model file (NBmodel.txt) to do an in-memory join. This file can be added to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]



### Q3 Student Answers:

> __a)__ 
- The keys in NBmodel are the city names. The first column represent the frequency of each word appearing in CLASS_0. The second column represents the frequency of each word appearing in CLASS_1. The third column represents the conditional probability of each word given CLASS_0 ($P(word|CLASS=0)$). The forth column represents the conditional probability of each word given CLASS_1 ($P(word|CLASS=1)$)
- For row:ClassPriors	1.0,3.0,0.25,0.75. The values are frequency of CLASS_0, CLASS_1, probability of CLASS_0, probability of CLASS_1.

Run these cells to create the example corpus and model.

In [98]:
%%writefile model/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Writing model/chineseTrain.txt


In [99]:
%%writefile model/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Writing model/chineseTest.txt


In [100]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Writing NBmodel.txt


In [105]:
# part a
# TODO: load the data files into HDFS
# write your commands below
!hdfs dfs -copyFromLocal model/chineseTrain.txt {HDFS_DIR}/chineseTrain.txt
!hdfs dfs -copyFromLocal model/chineseTest.txt {HDFS_DIR}/chineseTest.txt
!hdfs dfs -copyFromLocal NBmodel.txt {HDFS_DIR}/NBmodel.txt

In [108]:
!hdfs dfs -ls {HDFS_DIR}

Found 6 items
-rw-r--r--   1 root hadoop        303 2024-02-07 23:35 /user/root/HW2/NBmodel.txt
-rw-r--r--   1 root hadoop        119 2024-02-07 23:35 /user/root/HW2/chineseTest.txt
-rw-r--r--   1 root hadoop        107 2024-02-07 23:34 /user/root/HW2/chineseTrain.txt
drwxr-xr-x   - root hadoop          0 2024-02-07 04:03 /user/root/HW2/eda-output
drwxr-xr-x   - root hadoop          0 2024-02-07 04:37 /user/root/HW2/eda-sort-output
-rw-r--r--   1 root hadoop     204559 2024-02-07 02:08 /user/root/HW2/enron.txt


Your work for `part b` starts here:

In [114]:
# part b - do your work in model/classify_mapper.py first, then run this cell.
!chmod a+x model/classify_mapper.py

In [124]:
# part b - unit test model/classify_mapper.py (RUN THIS CELL AS IS)
!cat model/chineseTest.txt | model/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [133]:
# part b - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

Deleted /user/root/HW2/chinese-output


In [134]:
# part c - write your Hadooop streaming job here
!hadoop jar {JAR_FILE} \
  -files /model/classify_mapper.py,NBmodel.txt \
  -mapper /model/classify_mapper.py \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.10.2.jar] /tmp/streamjob7577554883876162334.jar tmpDir=null
24/02/08 01:54:11 INFO client.RMProxy: Connecting to ResourceManager at hw2hw2cluster-m/10.128.0.2:8032
24/02/08 01:54:12 INFO client.AHSProxy: Connecting to Application History server at hw2hw2cluster-m/10.128.0.2:10200
24/02/08 01:54:12 INFO client.RMProxy: Connecting to ResourceManager at hw2hw2cluster-m/10.128.0.2:8032
24/02/08 01:54:12 INFO client.AHSProxy: Connecting to Application History server at hw2hw2cluster-m/10.128.0.2:10200
24/02/08 01:54:12 INFO mapred.FileInputFormat: Total input files to process : 1
24/02/08 01:54:13 INFO mapreduce.JobSubmitter: number of splits:10
24/02/08 01:54:13 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1707263943154_0013
24/02/08 01:54:13 INFO conf.Configuration: resource-types.xml not found
24/02/08 01:54:13 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
24/02/08 01:54:13 INFO resource.Re

In [136]:
# part c - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > model/chineseResults.txt

In [137]:
# part c - take a look (RUN THIS CELL AS IS)
!cat model/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set for parts b) and c):</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 4: Naive Bayes Training
Previously, you trained a model by hand. In this question, you'll develop the code to do the same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process, review Q3 and write down the pieces of information needed to encode a Multinomial Naive Bayes model. We will retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totaled up _before_ you start estimating the class conditional probabilities for each word. 

**[IMPORTANT NOTE:]** you only need to use 1 reducer for this question. 


### Tasks:
* __a) make a plan:__  Read the docstrings (comments) in __`model/train_mapper.py`__ and fill in the docstrings for __`model/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`email.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement mapper and reducer:__ Complete the code in __`model/train_mapper.py`__ and __`model/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own choosing). When you are satisfied with your Python code design, run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 6.


* __c) short answer:__ You saw in the previous question that adding Laplace smoothing (where the smoothing parameter $k=1$) makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` review the smoothing equation again].


* __d) implement a smoothing reducer:__ Complete the code in __`model/train_reducer_smooth.py`__ to implement the algorithm with LaPlace smoothing. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above. 

    - [`HINT:` Don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications]. 


### Q4 Student Answers:

> __c)__ The number of unique words in the vocabulary.


In [145]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x model/train_mapper.py
!chmod a+x model/train_reducer.py

!echo "=========== REDUCER DOCSTRING ============"
!head -n 8 model/train_reducer.py | tail -n 6

=========== REDUCER DOCSTRING ============
Reducer aggregates word counts by class and emits frequencies.

INPUT:                                                     
    word \t class0_partialCount,class1_partialCount  
OUTPUT:
    word \t ham_count,spam_count,P(word|ham),P(word|spam)


__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [189]:
# part b - write a unit test for your mapper here
!echo '0001	0	 christmas tree farm pictures	NA\n0001	0	 christmas tree farm pictures	NA\n0001	0	 christmas tree farm pictures	NA\n0001	1	 christmas tree farm pictures	NA'| /model/train_mapper.py | column -t

#totals      15,5
ClassPriors  3,1
christmas    3,1
tree         3,1
farm         3,1
pictures     3,1
na           3,1


In [177]:
# part b - write a unit test for your reducer here
!echo '#totals	4,1\n#totals	4,1\nClassPriors	2,1\nClassPriors	2,1\nchristmas	4,1\nchristmas	4,0\ntree	2,0\ntree	2,1'| /model/train_reducer.py |column -t

christmas    8,1,1.0,0.5
tree         4,1,0.5,0.5
ClassPriors  4,2,0.6666666666666666,0.3333333333333333


In [188]:
# part b - write a systems test for your mapper + reducer together here
!echo '0001	0	 christmas tree farm pictures	NA\n0001	0	 christmas tree farm pictures	NA\n0001	0	 christmas tree farm pictures	NA\n0001	1	 christmas tree farm pictures	NA' | /model/train_mapper.py | /model/train_reducer.py |  column -t

christmas    3,1,0.2,0.2
tree         3,1,0.2,0.2
farm         3,1,0.2,0.2
pictures     3,1,0.2,0.2
na           3,1,0.2,0.2
ClassPriors  3,1,0.75,0.25


In [190]:
!hdfs dfs -ls {HDFS_DIR}

Found 7 items
-rw-r--r--   1 root hadoop        303 2024-02-07 23:35 /user/root/HW2/NBmodel.txt
drwxr-xr-x   - root hadoop          0 2024-02-08 01:54 /user/root/HW2/chinese-output
-rw-r--r--   1 root hadoop        119 2024-02-07 23:35 /user/root/HW2/chineseTest.txt
-rw-r--r--   1 root hadoop        107 2024-02-07 23:34 /user/root/HW2/chineseTrain.txt
drwxr-xr-x   - root hadoop          0 2024-02-07 04:03 /user/root/HW2/eda-output
drwxr-xr-x   - root hadoop          0 2024-02-07 04:37 /user/root/HW2/eda-sort-output
-rw-r--r--   1 root hadoop     204559 2024-02-07 02:08 /user/root/HW2/enron.txt


In [200]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/unsmoothed-chinese-output

Deleted /user/root/HW2/unsmoothed-chinese-output


In [201]:
# part b - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -files /model/train_mapper.py,/model/train_reducer.py \
  -mapper /model/train_mapper.py \
  -reducer /model/train_reducer.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/unsmoothed-chinese-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.10.2.jar] /tmp/streamjob5267554845565022786.jar tmpDir=null
24/02/09 00:46:54 INFO client.RMProxy: Connecting to ResourceManager at hw2hw2cluster-m/10.128.0.2:8032
24/02/09 00:46:54 INFO client.AHSProxy: Connecting to Application History server at hw2hw2cluster-m/10.128.0.2:10200
24/02/09 00:46:54 INFO client.RMProxy: Connecting to ResourceManager at hw2hw2cluster-m/10.128.0.2:8032
24/02/09 00:46:54 INFO client.AHSProxy: Connecting to Application History server at hw2hw2cluster-m/10.128.0.2:10200
24/02/09 00:46:55 INFO mapred.FileInputFormat: Total input files to process : 1
24/02/09 00:46:55 INFO mapreduce.JobSubmitter: number of splits:10
24/02/09 00:46:55 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1707263943154_0015
24/02/09 00:46:56 INFO conf.Configuration: resource-types.xml not found
24/02/09 00:46:56 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
24/02/09 00:46:56 INFO resource.Re

In [202]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/unsmoothed-chinese-output/part-0000* > /unsmoothed-chinese-results.txt

In [204]:
# part b - print your model so that we can confirm that it matches expected results
!cat /unsmoothed-chinese-results.txt | column -t

beijing      0,1,0.0,0.125
chinese      1,5,0.3333333333333333,0.625
japan        1,0,0.3333333333333333,0.0
macao        0,1,0.0,0.125
shanghai     0,1,0.0,0.125
tokyo        1,0,0.3333333333333333,0.0
ClassPriors  1,3,0.25,0.75


In [205]:
!cat model/chineseTrain.txt | column -t

D1  1  Chinese  Beijing  Chinese
D2  1  Chinese  Chinese  Shanghai
D3  1  Chinese  Macao
D4  0  Tokyo    Japan    Chinese


__`part d starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [209]:
!chmod a+x /model/train_reducer_smooth.py

In [210]:
# part d - write a unit test for your NEW reducer here
!echo '#totals	4,1\n#totals	4,1\nClassPriors	2,1\nClassPriors	2,1\nchristmas	4,1\nchristmas	4,0\ntree	2,0\ntree	2,1'| /model/train_reducer_smooth.py |column -t

christmas    8,1,0.643,0.25
tree         4,1,0.357,0.25
ClassPriors  4,2,0.667,0.333


In [211]:
# part d - write a systems test for your mapper + reducer together here
!echo '0001	0	 christmas tree farm pictures	NA\n0001	0	 christmas tree farm pictures	NA\n0001	0	 christmas tree farm pictures	NA\n0001	1	 christmas tree farm pictures	NA' | /model/train_mapper.py | /model/train_reducer_smooth.py |  column -t

christmas    3,1,0.19,0.182
tree         3,1,0.19,0.182
farm         3,1,0.19,0.182
pictures     3,1,0.19,0.182
na           3,1,0.19,0.182
ClassPriors  3,1,0.75,0.25


In [ ]:
# part d - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/smoothed-chinese-output

In [212]:
# part d - write your hadoop streaming job
!hadoop jar {JAR_FILE} \
  -files /model/train_mapper.py,/model/train_reducer_smooth.py \
  -mapper /model/train_mapper.py \
  -reducer /model/train_reducer_smooth.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/smoothed-chinese-output \
  -numReduceTasks 1 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.10.2.jar] /tmp/streamjob4877839433064442159.jar tmpDir=null
24/02/09 01:02:44 INFO client.RMProxy: Connecting to ResourceManager at hw2hw2cluster-m/10.128.0.2:8032
24/02/09 01:02:44 INFO client.AHSProxy: Connecting to Application History server at hw2hw2cluster-m/10.128.0.2:10200
24/02/09 01:02:44 INFO client.RMProxy: Connecting to ResourceManager at hw2hw2cluster-m/10.128.0.2:8032
24/02/09 01:02:44 INFO client.AHSProxy: Connecting to Application History server at hw2hw2cluster-m/10.128.0.2:10200
24/02/09 01:02:45 INFO mapred.FileInputFormat: Total input files to process : 1
24/02/09 01:02:45 INFO mapreduce.JobSubmitter: number of splits:10
24/02/09 01:02:45 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1707263943154_0016
24/02/09 01:02:46 INFO conf.Configuration: resource-types.xml not found
24/02/09 01:02:46 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
24/02/09 01:02:46 INFO resource.Re

In [213]:
# part d - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/smoothed-chinese-output/part-0000* > /smoothed-chinese-results.txt

In [214]:
!cat /smoothed-chinese-results.txt | column -t

beijing      0,1,0.111,0.143
chinese      1,5,0.222,0.429
japan        1,0,0.222,0.071
macao        0,1,0.111,0.143
shanghai     0,1,0.111,0.143
tokyo        1,0,0.222,0.071
ClassPriors  1,3,0.25,0.75


### Congratulations, you have completed HW2! 

### Submission Instructions
You will need to submit a zip file to **Gradescope** containing the following files:
- This notebook HW2.ipynb
- From Question 1, eda/mapper.py, eda/reducer.py, and optionally mapper-by-class.py
- From Questions 3-4, model/classify_mapper.py, model/train_mapper.py, model/train_reducer.py, model/train_reducer_smooth.py